Source publication for dataset: 
https://www.nature.com/articles/s41467-018-02866-0

In [1]:
import anndata as ad
import numpy as np
import pandas as pd
from copy import deepcopy

In [2]:
n_cells = 456
n_genes = 100
PWscores = np.zeros((n_cells, n_genes, n_genes))

for i in range(n_cells):
    path = "./../data/TEsmESC/geneXgene{i}.csv".format(i=(i+1))
    df_temp = pd.read_csv(path)
    PWscores[i, :, :] = np.copy(df_temp.values)
    

TODO: <br />
Make the graph! We got cells, and we got PW gene scores for all the cells! <br />
What does a high score mean? It means that within the neighbourhood of that cell, the first gene is informative of the other gene! <br />
Each entry of the matrix: PWscores[i, j, k] represents the transfer entropy score of j → k in the neighbourhood of cell i. <br />
Why's that significant? How does that help anyone?
--> Read up on that! <br/> <br/>

Steps: <br/>
1. Get the OG data and remake the trajectory by just putting the most similar cells side-by-side <br/>
2. Do the same with the PW network and see how the two compare <br/>
(first two steps are just to get a sense of the cell manifold)<br/>
3. Find high cost PATHS in each cell, i.e. represent each cell as an interaction network and find the highest entropy paths through it <br/>
4. Use these as the edges of your hypergraph, and do this for all cells <br/>
(it makes sense because we're interested in what gene pathways have high TE scores across cells)<br/>
5. Spectral clustering hopefully reveals some developmentally critical genes<br/>
(keep in mind this is a dataset of transcription factors!)<br/> 


Initialize the path(s) with the maximum value(s) <br/>
    - decide which values to keep <i>relative</i> to the <i>original maximum</i> value

In [4]:
# Remember, the entries in the matrix are weights for edges... what you want to ultimately output are the genes involved in the edges!
# row/col index = row->col edge, i.e. entropy between gene_row->gene_col (TODO: or the reverse)
# so you wanna output the row->col indices, and eventually map those back to gene names! 

# The informative gene pathway extraction algorithm
(For now implemented as just an easy intuitive iterative solution, maybe later will attempt a numpy/C++ acceleration!)

In [8]:
def get_hyperedge_set(cell_index):

    cell_nw = PWscores[cell_index, :, :]
    
    # init
    paths = []
    max_itrs = n_genes
    itrs = 0
    tolerance = np.max(cell_nw) * 0.2 # all scores within whatever % of the max
    max_path_len = 0
    max_path_len_prev = -1

    for i in range(n_genes): #row is the predecessor node
        index_max = np.argmax(cell_nw[i, :]) # index of max column is the successor (incident) node
        
        if cell_nw[i, index_max] > tolerance:
            paths.append([i, index_max])

    # update
    while (max_itrs > itrs) and (max_path_len > max_path_len_prev):
        max_path_len_prev = max_path_len
        new_paths = deepcopy(paths) 
        for p in range(len(paths)):
            path = paths[p]
            if len(path) >= max_path_len:
                row = path[-1]
                index_max = np.argmax(cell_nw[row, :])
                
                if (cell_nw[row, index_max] > tolerance) and (index_max not in new_paths[p]):
                    new_paths[p].append(index_max)

        max_path_len = max( len(x) for x in new_paths )
        itrs += 1
        paths = deepcopy(new_paths)
    
    return paths

# Creation of the hypergraph

In [11]:
# TODO

# Spectrally clustering the hypergraph
(hopefully revealing the informative gene pathways)

In [ ]:
# TODO